# SNP-Dataset
실제 MRI-Dataset을 활용하기 위하여 <a href="http://adni.loni.usc.edu/">ANDI</a>에서 Dataaset을 다운받았다.  

SNP-Dataset은 위의 링크의 ADNI에서 제공하는 Plink File Format을 사용하였습니다.

<span style="color:red;">Plink의 File과 결과가 Size가 크기 때문에, GitHub에 올리지 못하였고, 과정에 대해서만 적었습니다.</span>

각각의 과정은 다음과 같은 Preprocessing을 거치게 됩니다.

1. ADNI1 SNPs Plink Dataset Download
2. ADNI1 SNPs Quality Control
 - NL_MCI or NL_AD => Method 1
 - NL_MCI_AD => Method2
 
3. Method1
 - Subject Cluster
 - Filtering SNPs: 1) Sex Check 2) MAF: 5%, 3) Missing Value: 10%
 - Logistic Regression => P-Value Sorting => Select Snps with low p-value
 - Add APOE Gene SNPs(rs429358)
 
4. Method2
 - Filtering SNPs: 1) Sex Check 2) MAF: 1%, 3) Missing Value: 5% 4) Hardy-Weinberg equilbrium: p<1E-07 5) Linkage Disequilibrium
 - One-Way ANOVA (Python) => P-Value Sorting => Select Snps with low p-value
 
각각의 Method를 나눈 이유는 다음과 같습니다.
- Method1: Multi-Modality Disease Modeling via Collective Deep Matrix Factorization (https://dl.acm.org/doi/pdf/10.1145/3097983.3098164)
- Method2: A large scale multivariate parallel ICA method reveals novel imaging–genetic relationships for Alzheimer's disease in the ADNI cohort's Method (https://pubmed.ncbi.nlm.nih.gov/22245343/)

위의 각각의 논문에서 전처리한 방식에 따라서 따라하였습니다.  
위와 같은 방식을 사용하는 이유는 SNPs는 현재 60만개 이상이므로, 이 중 해당 Prediction하고 싶은 질병과 관련이 있는 SNPs(DEG)를 뽑아서 Model에 넣기 위함입니다.

각각의 Plink를 통한 Filtering의 Argument는 현재 Project와 맞지않아서, 설명을 생략하겠습니다.

**위의 결과는 모든 Sample에 대하여 수행한 결과의 Figure입니다. 현재 Model에 들어가는 SNPs같은 경우, 모든 Sample에 대한 결과가 아닌 Sample * 0.9로서 Model의 Trainning에 들어가는 Sample과 동일하게 DEG를 Select하였습니다. 모든 Sampel에 대한 P-value를 구한 뒤 DEG를 뽑는 것은, Machine Learning에서 Trainning Dataset과 Test Dataset을 나누는 이유과 같습니다.**

## NL vs MCI Result
![png](./image/8.png)

위의 사진은 모든 NL vs MCI에 대한 Logistic Regression의 P-value를 -log10 Scale한 결과 입니다.  
**Alzihmer Disease와 관련이 높은 APOE Gene에 관련된 SNPs가 가장 낮은 P-value를 가지는 것을 확인할 수 있습니다.**

위의 결과를 확인하기 위한 PLink의 Command는 다음과 같습니다.

1) Subject Cluster: <code>plink --bfile ADNI1 --keep 90_subject.txt --make-bed --out ADNI1_NL_MCI_90</code>  
2) SNPs Quality Control: <code>plink --bfile ADNI1_NL_MCI_90 --geno 0.1 --maf 0.05 --recode --make-bed --out Filtering_90 --noweb</code>
 - MAF: 5%
 - Missing Value: 10%
 
3) Rogistic Regression: <code>plink --bfile Filtering_90 --logistic --pheno 90_subject_label.txt --allow-no-sex --ci 0.95 --out NL_MCI_ADNI_1_Result_90</code>
4) DEG SNP's Select: <code>plink --bfile Filtering --extract SNP_LIST_90.txt --make-bed --out Final_List_90 --noweb
</code>
5) Convert to VCF File Format: <code>plink --bfile Final_List_90 --recode vcf --out VCF_90</code>  
6) VCF File -> Save DataFrame(.csv File Format) by Using R

VCF File은 Python에서 처리하는 Tool이 없어서 R에서 csv File Format으로 변경하였습니다. (R Code는 따로 첨부하지 않았습니다.)

## NL vs AD Result
![png](./image/9.png)

위의 사진은 모든 NL vs MCI에 대한 Logistic Regression의 P-value를 -log10 Scale한 결과 입니다.  
**Alzihmer Disease와 관련이 높은 APOE Gene에 관련된 SNPs가 가장 낮은 P-value를 가지는 것을 확인할 수 있습니다.**

위의 결과를 확인하기 위한 PLink의 Command는 다음과 같습니다.

1) Subject Cluster: <code>plink --bfile ADNI1 --keep 90_subject.txt --make-bed --out ADNI1_NL_AD_90</code>  
2) SNPs Quality Control: <code>plink --bfile ADNI1_NL_AD_90 --geno 0.1 --maf 0.05 --recode --make-bed --out Filtering_90 --noweb</code>
 - MAF: 5%
 - Missing Value: 10%
 
3) Rogistic Regression: <code>plink --bfile Filtering_90 --logistic --pheno 90_subject_label.txt --allow-no-sex --ci 0.95 --out NL_AD_ADNI_1_Result_90</code>
4) DEG SNP's Select: <code>plink --bfile Filtering --extract SNP_LIST_90.txt --make-bed --out Final_List_90 --noweb
</code>
5) Convert to VCF File Format: <code>plink --bfile Final_List_90 --recode vcf --out VCF_90</code>  
6) VCF File -> Save DataFrame(.csv File Format) by Using R

VCF File은 Python에서 처리하는 Tool이 없어서 R에서 csv File Format으로 변경하였습니다. (R Code는 따로 첨부하지 않았습니다.)

**NL vs MCI 와 NL vs AD의 결과를 살펴보게 되면, NL vs AD에서의 APOE에 관련된 SNPs가 P-value가 훨씬 낮은 것을 확인할 수 있다. => 기존에 Alzhimer Disease의 영향을 미친다고 알려진 SNPs가 유의하게 나타나고, NL vs AD에서 더 낮은 것으로서 SNPs의 Labeling이 잘못되지 않은 것을 확인하였습니다.**

## NL vs MCI vs AD

1) Sex Check: <code>plink --bfile ADNI_cluster_01_forward_757LONI --check-sex</code> => **2 Subject 073_S_0909, 130_S_1201**에서 PROBLEM이 발생한 것을 확인하였다.  

2) Quality Contorl 1: <code>plink --bfile filter_sex --geno 0.05 --maf 0.01 --hwe 0.0000001 --recode --out quality --noweb</code> 
 - MAF: 1%
 - Missing Value: 5%
 - HWE: p<1E-07
 
3) Quality Contorl 2: <code>plink --bfile quality --noweb --indep-pairwise 100 5 0.95 --out LD</code>
 - LD

4) Convert to VCF File Format: <code>plink --bfile LD --recode vcf --out VCF</code>

5) VCF File -> Save DataFrame(.csv File Format) by Using R

6) One-Way ANOVA (Python)

NL vs MCI vs AD의 경우에는 Plink에서 One-Way-ANOVA를 지원하지 않아여 Python으로서 구현하였습니다. 따라서 Figure가 존재하지 않습니다.  
ANOVA의 경우 Python Code로서 작성하였으나, 현재 Project의 목적과 맞지 않으므로 올리지 않았습니다.